# [모듈 0.0] Code Pipeline 및 SageMaker 모델 훈련 파이프라인 생성 플로우

아래의 내용은 "Code Pipeline 및 SageMaker 모델 훈련 파이프라인 생성 플로우" 의 실행하기 위한 파일 구조 및 실행 단계를 설명 합니다. 
- 코드는 실행을 하면 문제 없이 됩니다. 하지만 본인의 모델을 가지고 작업시에는 아래의 "방법" 을 잘 숙지해야만 합니다. 


# A. 중요 파일 구조
- "4_sm-train-codepipeline" 의 폴더 아래의 파일 구조를 유심히 보시면 됩니다.


```
0_Setup_Environment
1_Train
2_Inference
3_MLOps
 1_sm_training_pipeline
  |-src
 2_sm_serving_pipeline
 3_hello-codepipeline
 4_sm-train-codepipeline
 |-0.0.create-pipeline.py-buildspec.yml.ipynb
 |-1.0.Create_Config.ipynb
 |-1.1.create_codecommit.ipynb
 |-2.1.build_project.ipynb
 |-3.1.pipeline_project.ipynb
 |-4.1.Cleanup.ipynb
 |-src
 | |-sm_pipeline_train_config.json
 | |-code_pipeline_train_config.json
 |-codecommit
 | |-codebuild-buildspec.yml
 | |-pipelines
 | | |-upload_code.py
 | | |-_utils.py
 | | |-__version__.py
 | | |-run_pipeline.py
 | | |-get_pipeline_definition.py
 | | |-ncf
 | | | |-src
 | | | | |-sm_pipeline_train_config.json
 | | | | |-code_pipeline_train_config.json
 | | | |-pipeline.py
 | | | |-code_location.json
 | |-sagemaker-pipelines-project.ipynb
 | |-code_location.json
 | |-setup.py
 | |-setup.cfg
5_sm-serving-codepipeline
```

# B 모델 훈련 파이프라인 흐름


## B.1.상위 레벨의 전체적인 생성 순서 
- 아래와 같은 (1) ~ (4) 은 이미 노트북의 코드로 구현이 된 상태 입니다. 추후에 본인의 모델로 작업시에 코드를 참고 하시면 됩니다.
- (5) ~ (8) 은 직접 노트북을 실행하여 CodePipeline 을 통하여 SageMaker Pipeline 을 실행 합니다.
- (8) 이 실행 후에 (9), (10) 이 자동으로 실행 됩니다

![how-to-convert-codepipeline.png](img/how-to-convert-sm-codepipeline.png)


## B.2. CodePipeline 에서 SageMaker Pipeline 실행
- CodePipeline 생성시에 codepipeline_config.json 을 파라미터를 통하여 환경 변수를 세팅 합니다.
- codebuild-buildspec.yml 은 run-pipeline.py 를 실행하여 SageMaker Pipeline 을 동작 시킵니다.

![sm_pipeline_execution_flow.png](img/sm_pipeline_execution_flow.png)

# C. 작업 순서
1.  훈련 스크립트 복사. 1_sm_training_pipeline 에서 사용한 코드 파일을 CodeCommit repo 에 저장하기 위해 임시로 복사를 하는 과정 임
    - 원본: 1_sm_training_pipeline/src 
    - 타겟: codecommit/pipelines/ncf/src
2.  setup.py 수정
    - codecommit/pipelines/ncf/src 폴더가 Python Package 생성시 포홤되게 수정.
3.  1.0.Create_Config.ipynb
    - src 폴더에 아래 두개의 설정 파일 저장 하고, 
        -sm_pipeline_train_config.json
        -code_pipeline_train_config.json
        - 아래의 파일에서 사용 함.
        - 1.1.create_codecommit.ipynb
        - 2.1.build_project.ipynb
        - 3.1.pipeline_project.ipynb
        - 4.1.Cleanup.ipynb
    - 위의 두개의 설정 파일을 codecommit/pipelines/ncf/src 에 복사
        - Code Pipeline 에 의해서 SageMaker Pipeline 실행시 사용 함.
4.  codecommit/pipelines/ncf/pipeline.py
    - SageMaker Pipeline 의 구현 코드 
5.  codecommit/sagemaker-pipelines-project.ipynb
    - 위의 4의  codecommit/pipelines/ncf/pipeline.py 의 테스트를 위한 노트북
    - 원래 템플랫의 origin-sagemaker-pipelines-project.ipynb 파일을 수정한 버전
6.  codecommit/pipelines/run_pipeline.py
    - pipeline = get_pipeline() 을 통해서 pipeline 오브젝트를 받아 실제로 pipeline.start() 실행 함.
7.  1.1.create_codecommit.ipynb
    - codecommit repo 를 생성하고, 소스 코드를 저장함.
8.  2.1.build_project.ipynb
    - code build project 을 생성. 환경 변수를 정의 함.
9.  codecommit/codebuild-buildspec.yml 수정
    - 7번의 환경 변수를 이용하여 "4.  codecommit/pipelines/ncf/pipeline.py" 를 실행하는 명령어를 기술 함.
10.  3.1.pipeline_project.ipynb
    - 6. codecommit, 7. codebuild 를 연결하여 code pipeline 을 만들고 실행 함.
11. 4.1.Cleanup.ipynb
    - 리소스 정리 노트북

# D. 상세 사항

# 3. pipeline.py
- 위치
    - codecommit/pipelines/ncf/pipeline.py
- 3_MLOps/1_sm_training_pipeline/3.1.NCF-Training-Pipeline.ipynb 의 내용을 수정해서 만듦

## 3.1 get_pipeline(()

```
def get_pipeline(
    project_prefix,
    region,
    role=None, # SAGEMAKER_PIPELINE_ROLE_ARN 이 넘어옴.
    default_bucket=None,
    model_package_group_name= None,
    pipeline_name= None
):
```
- 위의 파라미터는 codebuild-buildspec.yml 의 run-pipeline 명령어의 kwargs 파라미터의 내용 임.


## 3.2  람다 스템을 위한 lambda_role 을 SAGEMAKER_PIPELINE_ROLE_ARN 로 대체 함.
- SAGEMAKER_PIPELINE_ROLE_ARN 은 아래와 같은 정책 및 신뢰 관계가 있어야 함.
- 필요 정책
    - AWSLambda_FullAccess
    - AmazonSageMakerFullAccess
- Trust Relationship    
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```

## 3.3 람다 스텝 코드 위치 복사
- 아래와 같이 3_MLOps/3_sm-train-codepipeline/codecommit/pipelines/ncf 에 iam_repackage_model_artifact.py 복사

![iam_repackage_model_artifact.png](img/iam_repackage_model_artifact.png)

### 3.4 람다 스텝 변경
- script 에 repackage_lambda_script_path 의 절대 경로 지정

``` python
    repackage_lambda_script_path = f'{BASE_DIR}/iam_repackage_model_artifact.py'

    func_repackage_model = Lambda(
        function_name=function_name,
        execution_role_arn=role,
        script=repackage_lambda_script_path,        
        handler="iam_repackage_model_artifact.lambda_handler",
    )

```

## 3.5 Model Package Group Name 변경
- model_package_group_name 를 함수 인자로 받아서 사용 함.

```python
    model_package_group_input_dict = {
     "ModelPackageGroupName" : model_package_group_name,
     "ModelPackageGroupDescription" : "Sample model package group"
    }

```

# 6. codecommit/pipelines/run_pipeline.py 작성
- pipelines/ncf/src/sm_pipeline_train_config.json 설정 파일에서 변수를 가져와서 파이프라인 변수로 제공

```python
execution = pipeline.start(
    parameters=dict(
        InputData= s3_input_data_uri,
        training_instance_type = training_instance_type,
        training_instance_count = training_instance_count,
        ModelApprovalStatus = ModelApprovalStatus,            
        inference_image_uri = inference_image_uri,     
    )
)        

```

# 9. codebuild-buildspec.yml 파일
- 위치: 
    - codecommit/codebuild-buildspec.yml

```yml
version: 0.2

phases:
  install:
    runtime-versions:
      python: 3.8
    commands:
      - pip install --upgrade --force-reinstall . "awscli>1.20.30"
  
  build:
    commands:
      - python pipelines/upload_code.py --code-repository-name $code_repository_name --bucket $TEMPLATE_BUCKET
      - export PYTHONUNBUFFERED=TRUE
      - |
        run-pipeline --module-name pipelines.ncf.pipeline \
          --role-arn $SAGEMAKER_PIPELINE_ROLE_ARN \
          --kwargs "{\"region\":\"${AWS_REGION}\",\"role\":\"${SAGEMAKER_PIPELINE_ROLE_ARN}\",\"default_bucket\":\"${TEMPLATE_BUCKET}\",\"pipeline_name\":\"${SAGEMAKER_PIPELINE_NAME}\",\"model_package_group_name\":\"${model_package_group_name}\",\"project_prefix\":\"${project_prefix}\"}"
      - echo "Create/Update of the SageMaker Pipeline and execution completed."


```
- 이 파일에서 사용된 환경 변수는 codebuild_project 생성시에 환경 변수에서 가져오게 됨. 
     - 대표적으로 model_package_group_name 는 Model Registry 경로 임. 이 값을 변경하기 위해서는 codebuild_project 의 환경 변수를 수정해야 함.

## 9.1 코드를 S3에 업로드

```
- python pipelines/upload_code.py --code-repository-name $code_repository_name --bucket $TEMPLATE_BUCKET
```      
- 위의 코드를 실행하면 code_location.json 파일이 생성 됨, 아래와 같이 코드가 압축이 되어서 S3에 source.tar.gz 파일이 저장 됨.
    - Repackage 람다 스텝에서 사용 됨.
```
{
    "s3_location": "s3://sathiyajith"
}
```

## 9.2 파이프라인 실행

```Yml
run-pipeline --module-name pipelines.ncf.pipeline \
          --role-arn $SAGEMAKER_PIPELINE_ROLE_ARN \
          --kwargs "{\"region\":\"${AWS_REGION}\",\"role\":\"${SAGEMAKER_PIPELINE_ROLE_ARN}\",\"default_bucket\":\"${TEMPLATE_BUCKET}\",\"pipeline_name\":\"${SAGEMAKER_PIPELINE_NAME}\",\"model_package_group_name\":\"${model_package_group_name}\",\"project_prefix\":\"${project_prefix}\"}"
```
- kwargs 이후의 파라미터는 파싱이 되어서 pipeline.py 의 get_pipeline() 함수의 파라미터로 전달이 됨.

### run-pipeline.py 의 실행 순서
- pipeline = _utils.get_pipeline_driver(args.kwargs)
    - get_pipeline(**kwargs)
- pipeline.upstart()    
- pipeline.start()
